# Henting, lesing, og rensing av Trondheim forecast

### Utviklingsmiljøet:
Arbeidet med utviklingen av gruppe 84 sin miljødataaplikasjon har hovedsakelig foregått på brukeren Edi Galane sin github og VScode. De andre brukerene har bidrat med å skrive ulike deler av koden og funksjonaliteten, men endringene til selve koden har blitt utført gjennom Edi Galane din datamaskin. 

### Valg av API:
Datagrunnlaget for dette prosjektet har vært basert på MET.no (Meteorologisk institutt) sin åpne Locationforecast 2.0.

Det meteorologiske instituttet er en svært pålitelig kilde og Locationforecast 2.0 er et av deres høyest kurerte produkter. Til tross for at databasen er av høyest kvalitet, så er det nødvendig å utføre utforskende metoder for å sikre dens kvalitet og hente ut kun de nødvendige informasjonene. 

#### Import av nødvendige moduler og klasser:

In [ ]:
import os
import sys

# legger til src-mappen i systemstien
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

# import av egne moduler og klasser
from fetch_data import FetchData
from data_reader import DataLeser
from data_cleaning import DataRensing


#### Henting og lagring av nødvendig rådata:

In [ ]:
henter = FetchData()
henter.lagre_trondheim_forecast()

- lagre_trondheim_forecast() er en metode fra FetchData som har innebygd metoder for å hente en JSON-fil fra Locationforecast2.0 og flate den ut til en CSV-fil basert på parametrene for byen Trondheim.

#### Les og beskriv:

In [ ]:
leser = DataLeser()
df_raw = leser.les_csv("trondheim_forecast_uncleaned.csv")  
leser.beskriv_dataframe(df_raw, "trondheim_forecast_uncleaned.csv")

- Her ser vi de 5 første radene av det urensede datasettet. Neste steg er å velge ut de nødvendige kolonnene og rense dem.

#### Rense og lagre:
- Her tas data_rens metoden i bruk fra DataRensing klassen. Den inkluderer følgende metoder:
    - Brukerinteraktivitet for avrunding og filtrering av kolonner.
    - Bygging av en ny pandas DataFrame.
    - Fjerning av duplikater for å unngå overrepresentasjon av datapunkter. 
    - Median imputasjon for manglende verdier som bevarer semtraltendenser uten å påvirkes ekstreme verdier.
    - Bearbeiding av tidsstempler til lesbar datetime format.
    - Lagring av det nye pandas DataFrame til data mappen under "csv".
           

In [ ]:
renser = DataRensing(df_raw)

# Brukerinput
temp_input = input("Filtrer ut temperatur over (la stå tom for å hoppe over): ")
avrund_input = input("Vil du avrunde en kolonne? (la stå tom for å hoppe over): ")
desimal_input = input("Hvor mange desimaler? (standard = 1): ")

# Konversjon av input
temp_grense = float(temp_input) if temp_input else None
avrund_kol = avrund_input if avrund_input else None
desimaler = int(desimal_input) if desimal_input else 1

# Bruker data_rens metoden
df_cleaned = renser.data_rens( 
    temp_grense=temp_grense,
    avrund_kol=avrund_kol,
    desimaler=desimaler,
    filnavn="trondheim_forecast_cleaned.csv",
    undermappe="data/csv"
)

# Beskriv det rensede datasettet
leser.beskriv_dataframe(df_cleaned,trondheim_forecast_cleaned.csv)

Etter rensingen har vi ulike transformasjoner:
- Kolonnene [Tid, Temperatur, Fuktighet, Trykk, Vindhastighet] er bevart og omdøpt til lesbare navn. 
- Duplikater fjernes for å unngå at analysen blir skjevfordelt eller vektet feil.
- Manglende verdier inputeres som medianen siden denne ikke påvirkes av ekstreme uteliggere som kan oppdages i værdata grunnet dens urregelmessighet
- Resultatet er en DataFrame som lettere kan analyseres og modelleres. 

#### SQL-spørring
Vi har også implementert en valgfritt sql_utforsk metode i DataLeser. Men hva gjør SQL egentlig?
- Filtrerer data raskt basert på bestemte kriterier.
- Velger ut spesifikke kolonner og rader.
- Utfører aggregering som gjennomsnitt, maks, min, grupperinger osv...

In [ ]:
sql_query = """
SELECT
    Tid,
    Temperatur,
    Fuktighet,
    Trykk,
    Vindhastighet
FROM df
WHERE Temperatur > 10
ORDER BY Tid ASC
LIMIT 10
"""

print("Hent de 10 første radene med temperatur > 10 grader celcius")
try:
    df_sql = leser.sql_utforsk(df_cleaned, sql_query)
    display(df_sql)
except ValueError as e:
    print(f"Feil i SQL-spørring: {e}")

SQL-spørringen over henter de første 10 radene der temperaturen er over 10. Den viser tidspunktet, temperaturen, fuktigheten, trykket og vindhastigheten.
- Dette hjelper med å se hvordan ulike varaibler oppfører seg sammen under forhold over 10 grader celcius.